In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

### **Agentic Rag**

**질문에 따라 문서를 검색하여 답변하거나, 인터넷 검색 도구를 활용하여 답변하는 에이전트**

Agent 를 활용하여 RAG 를 수행한다면 Agentic RAG 라고 합니다. 

### **웹 검색도구: Tavily Search**

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

search  = TavilySearchResults(k=6)      # 검색 결과를 6개까지 가져오겠다

In [3]:
search.invoke("대구에 동성로 중앙떡볶이 전화번호는 무엇인가요?")

[{'url': 'https://www.siksinhot.com/P/410511',
  'content': '#동성로맛집 #동성로떡볶이 #대구중앙떡볶이 #대구맛집 #중앙떡볶이 중앙떡볶이 대구광역시 중구 동성로 2길 81 전화번호 : 0534247692 영업시간 : 11:30 ~ 19:00 (매주'},
 {'url': 'https://m.blog.naver.com/myclover87/222900193276',
  'content': '블로그\n\n카테고리 이동\n\n\n\n\nstar…☆\n\n대구 동성로 맛집 중앙떡볶이\n\n2022. 10. 28. 9:30\n\n대구광역시 중구 동성로2길 81 중앙떡볶이\n\n대구광역시 중구 동성로2길 80 2.28 기념중앙공원\n\n주소 : 대구 중구 동성로2길 81 중앙떡볶이\n\n영업시간 : 매일 11:30 - 19:30\n\n일요일 정기휴무(매달 1, 2, 3번째 일요일)\n\n전화번호 : 053-424-7692\n\n\u200b\n\n차량 이용 시\n\n네비에 동성로중앙주차장으로 검색하시면 돼요.\n\n30분 -1,000원\n\n60분 - 2,500원\n\n120분 - 5,500원\n\n240분 - 11,500원\n\n안녕하세요\n\n오늘 소개할 집은\n\n대구에서 너무 유명한 중앙떡볶이입니다.\n\n3대천왕에 나오기 전부터\n\n대구분들에겐 유명한 집이죠~\n\n\u200b\n\n참고로 중앙떡뽁이는 체인점이 없다 해요\n\n이름이 같다고 해서 똑같은 집은\n\n아니니 2.28기념 중앙공원 바로 앞에 있는\n\n집이 3대천왕에 나온 집입니다^^\n\n평일 점심이었는데도 [...] 식당 내부에 손님이 많아\n\n테이블 사진은 못 찍었네요~\n\n몇 년 전에는 주말에 갔던 터라\n\n줄이 너무 길어서 못 먹었었는데\n\n오늘은 평일이라!\n\n줄 안 사고 먹기 가능!\n\n이름을 비슷하게들 지으시니\n\n매장 곳곳에 이렇게\n\n체인점이 없다고 적어놓으셨어요.\n\n왼쪽은 줄 서서 기다리시는 분들\n\n인증샷을 식당 문

## **Retriever**

PDF 문서를 FAISS DB 에 저장하고 조회하는 retriever 를 생성합니다.

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyMuPDFLoader

In [5]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader('data/SPRI_AI_Brief_2023년12월호_F.pdf')
docs = loader.load()

In [6]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_documents = text_splitter.split_documents(docs)

In [7]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

In [8]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [9]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [10]:
# 문서에서 관련성 높은 문서를 가져옵니다.
retriever.invoke('삼성전자가 개발한 생성형 AI 관련 내용을 문서에서 찾아줘')

[Document(id='fc8a07bf-84a4-445d-b0ee-6e6fb23f97e8', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'page': 12}, page_content='SPRi AI Brief |  \n2023-12월호\n10\n삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개\nn 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스’를 공개\nn 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유\nKEY Contents\n£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원\nn 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, \n온디바이스에서 작동하도록 설계되어 외부로 사용자의 정

`create_retriever_tool()` 함수로 `retriever` 를 도구로 변환합니다.

In [11]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,              # 리트리버
    name='pdf_search',      # 도구의 이름
    
    # 도구에 대한 설명을 자세히 기입해야 합니다. 
    # LLM에게 알려주는 내용인데.. 이 툴을 언제 쓸지 언제 활용하면 좋을지를 영어로 작성합니다.
    description="use this tool to search information from the PDF document", 
)

### **Agent 가 사용할 도구 목록 정의**

`tools` 리스트는 `search`와 `retriever_tool`을 포함합니다.

In [12]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

### **Agent 생성**

Agent 가 활용할 LLM을 정의하고, Agent 가 참고할 Prompt 를 정의합니다.

In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [14]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

In [ ]:
prompt = ChatPromptTemplate.from_messages(                  # 프롬프트
    [
        (
            'system',
            'You are a helpful assistant. '
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ('placeholder', '{chat_history}'),
        ('human', '{input}'),
        ('placeholder', '{agent_scratchpad}'),
    ]
)

Tool Calling Agent 를 생성합니다.

In [16]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)       # tool calling agent 생성

agent 를 실행하는 AgentExecutor 를 생성합니다.

In [17]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(     
    agent=agent,                    # 각 단계에서 계획을 생성하고 행동을 결정하는 agent
    tools=tools,                    # agent 가 사용할 수 있는 도구 목록
    verbose=True,
    max_iterations=10,              # 최대 10번 까지 반복
    max_execution_time=10,          # 실행되는데 소요되는 최대 시간
    handle_parsing_errors=True      
)

### **에이전트 실행하기**

agent_executor 객체의 invoke() 메소드는 딕셔너리 형태의 인자를 받아 처리합니다. <br>
input 키에 사용자 질문을 값으로 할당한 딕셔너리를 인자로 전달합니다.<br>

In [18]:
answer = agent_executor.invoke({'input': '2024년 프로야구 플레이오프 진출한 5개의 팀을 검색하여 알려주세요'})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`


[{'url': 'https://namu.wiki/w/KBO%20%ED%94%8C%EB%A0%88%EC%9D%B4%EC%98%A4%ED%94%84/2024%EB%85%84', 'content': '... 년 한국시리즈가 마지막이다. 나눔 올스타 팀 중 LG 트윈스가 플레이오프 진출을, KIA 타이거즈가 한국시리즈 직행을 하면서 앞으로의 포스트시즌 경기 결과에 상관'}, {'url': 'https://filmk.tistory.com/entry/%ED%94%84%EB%A1%9C%EC%95%BC%EA%B5%AC-%ED%8F%AC%EC%8A%A4%ED%8A%B8%EC%8B%9C%EC%A6%8C-%EA%B2%BD%EA%B8%B0-%EC%9D%BC%EC%A0%95-%EB%B0%8F-%EA%B2%B0%EA%B3%BC-%EC%B4%9D%EC%A0%95%EB%A6%AC2024', 'content': '4) 한국시리즈 일정 ... 2024 한국 프로야구 플레이오프에서 승리한 팀은 KIA와 맞붙게 되며, 그 승부는 한국시리즈 진출을 결정짓는 중요한 대결이 될 것'}, {'url': 'https://aldalsuin.tistory.com/entry/2024-%ED%94%84%EB%A1%9C%EC%95%BC%EA%B5%AC-%ED%94%8C%EB%A0%88%EC%9D%B4%EC%98%A4%ED%94%84-9%EA%B0%9C-%EA%B5%AC%EB%8B%A8%EC%9D%98-%EC%B9%98%EC%97%B4%ED%95%9C-%EC%9A%B0%EC%8A%B9-%EA%B2%BD%EC%9F%81', 'content': '2024 프로야구 플레이오프에 진출한 팀들의 면면을 살펴보면, 각 팀마다 독특한 강점과 전략이 돋보입니다. 정규시즌 1위 팀부터 와일드카드 진출팀까지,'}

In [19]:
result = agent_executor.stream({'input': '2024년 프로야구 플레이오프 진출한 5개의 팀을 검색하여 알려주세요'})

for step in result:
    print('step:', step)
    print()
    
    if 'actions' in step:
        print('==' * 50)
        print(f'[actions]')

        for action in step['actions']:
            print(f"사용 도구 이름 : {action.tool}")
            print(f"입력 쿼리 : {action.tool_input}")
            print(f"Log: {action.log}")

        print('==' * 50)

    elif 'steps' in step:
        print('==' * 50)
        print(f'[steps]')

        for agent_step in step['steps']:
            print("Action:")
            print(f"  Tool: {agent_step.action.tool}")
            print(f"  Tool Input: {agent_step.action.tool_input}")
            print(f"  Log: {agent_step.action.log}")
            print(f"Observation: {agent_step.observation}")
        
        print('==' * 50)

    elif 'output' in step:
        print('==' * 50)
        print(f"[output]: {step['output']}")

        if 'messages' in step:
            for message in step['messages']:
                print(f"Message: {message.content}")
                
        print('==' * 50)



> Entering new None chain...
step: {'actions': [ToolAgentAction(tool='tavily_search_results_json', tool_input={'query': '2024년 프로야구 플레이오프 진출 팀'}, log="\nInvoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_2qhyTGmRXTY2bUi4XrAg3cAA', 'function': {'arguments': '{"query":"2024년 프로야구 플레이오프 진출 팀"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090'}, id='run-cd3e51ba-1b03-436d-9507-fd411bf6cb51', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024년 프로야구 플레이오프 진출 팀'}, 'id': 'call_2qhyTGmRXTY2bUi4XrAg3cAA', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'tavily_search_results_json', 'args': '{"query":"2024년 프로야구 플레이오프 진출 팀"}', 'id': 'call_2qhyTGmRXTY2bUi4XrAg3cAA', 'index': 0, 'type': 'tool_call

In [20]:
def tool_callback(tool) -> None:
    print(f'===== 도구 호출 =====')
    print(f"사용된 도구 이름 : {tool[0].tool}")
    print(f'===== 도구 호출 =====')

In [21]:
def obervation_callback(observation) -> None:
    print(f'\n===== 관찰 내용 =====')
    print(f"Observation: {observation[0].observation}")
    print(f'===== 관찰 내용 =====')

In [22]:
def result_callback(result: str) -> None:
    print(f'===== 최종 답변 =====')
    print(f"{result}")
    print(f'===== 최종 답변 =====')

In [23]:
result = agent_executor.stream({'input': '2024년 프로야구 플레이오프 진출한 5개의 팀을 검색하여 알려주세요'})

for step in result:
    if 'actions' in step:
        tool_callback(step['actions'])
    elif 'steps' in step:
        obervation_callback(step['steps'])
    elif 'output' in step:
        result_callback(step['messages'][0].content)          



> Entering new None chain...
===== 도구 호출 =====
사용된 도구 이름 : tavily_search_results_json
===== 도구 호출 =====

Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`


[{'url': 'https://namu.wiki/w/KBO%20%ED%94%8C%EB%A0%88%EC%9D%B4%EC%98%A4%ED%94%84/2024%EB%85%84', 'content': '... 년 한국시리즈가 마지막이다. 나눔 올스타 팀 중 LG 트윈스가 플레이오프 진출을, KIA 타이거즈가 한국시리즈 직행을 하면서 앞으로의 포스트시즌 경기 결과에 상관'}, {'url': 'https://filmk.tistory.com/entry/%ED%94%84%EB%A1%9C%EC%95%BC%EA%B5%AC-%ED%8F%AC%EC%8A%A4%ED%8A%B8%EC%8B%9C%EC%A6%8C-%EA%B2%BD%EA%B8%B0-%EC%9D%BC%EC%A0%95-%EB%B0%8F-%EA%B2%B0%EA%B3%BC-%EC%B4%9D%EC%A0%95%EB%A6%AC2024', 'content': '4) 한국시리즈 일정 ... 2024 한국 프로야구 플레이오프에서 승리한 팀은 KIA와 맞붙게 되며, 그 승부는 한국시리즈 진출을 결정짓는 중요한 대결이 될 것'}, {'url': 'https://aldalsuin.tistory.com/entry/2024-%ED%94%84%EB%A1%9C%EC%95%BC%EA%B5%AC-%ED%94%8C%EB%A0%88%EC%9D%B4%EC%98%A4%ED%94%84-9%EA%B0%9C-%EA%B5%AC%EB%8B%A8%EC%9D%98-%EC%B9%98%EC%97%B4%ED%95%9C-%EC%9A%B0%EC%8A%B9-%EA%B2%BD%EC%9F%81', 'content': '2024 프로야구 플레이오프에 진출한

In [24]:
from langchain_teddynote.messages import AgentStreamParser

agent_stream_parser = AgentStreamParser()           # 각 단계별 출력을 위한 파서 생성

result = agent_executor.stream(                     # 질의에 대한 답변을 스트리밍으로 출력 요청
    {'input': '2024년 프로야구 플레이오프 진출한 5개 팀을 검색하여 알려주세요.'}
)

for step in result:
    agent_stream_parser.process_agent_steps(step)   # 중간 단계를 parser 를 사용하여 단계별로 출력



> Entering new None chain...
[도구 호출]
Tool: tavily_search_results_json
query: 2024년 프로야구 플레이오프 진출 팀
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`




Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 플레이오프 진출 팀'}`


[{'url': 'https://namu.wiki/w/KBO%20%ED%94%8C%EB%A0%88%EC%9D%B4%EC%98%A4%ED%94%84/2024%EB%85%84', 'content': '... 년 한국시리즈가 마지막이다. 나눔 올스타 팀 중 LG 트윈스가 플레이오프 진출을, KIA 타이거즈가 한국시리즈 직행을 하면서 앞으로의 포스트시즌 경기 결과에 상관'}, {'url': 'https://filmk.tistory.com/entry/%ED%94%84%EB%A1%9C%EC%95%BC%EA%B5%AC-%ED%8F%AC%EC%8A%A4%ED%8A%B8%EC%8B%9C%EC%A6%8C-%EA%B2%BD%EA%B8%B0-%EC%9D%BC%EC%A0%95-%EB%B0%8F-%EA%B2%B0%EA%B3%BC-%EC%B4%9D%EC%A0%95%EB%A6%AC2024', 'content': '4) 한국시리즈 일정 ... 2024 한국 프로야구 플레이오프에서 승리한 팀은 KIA와 맞붙게 되며, 그 승부는 한국시리즈 진출을 결정짓는 중요한 대결이 될 것'}, {'url': 'https://aldalsuin.tistory.com/entry/2024-%ED%94%84%EB%A1%9C%EC%95%BC%EA%B5%AC-%ED%94%8C%EB%A0%88%EC%9D%B4%EC%98%A4%ED%94%84-9%EA%B0%9C-%EA%B5%AC%EB%8B%A8%EC%9D%98-%EC%B9%98%EC%97

In [25]:
result = agent_executor.stream({'input': '삼성전자가 자체 개발한 생성형 AI에 대한 정보를 문서에서 검색해서 알려주세요.'})

for step in result:
    if 'actions' in step:
        tool_callback(step['actions'])
    elif 'steps' in step:
        obervation_callback(step['steps'])
    elif 'output' in step:
        result_callback(step['messages'][0].content)          



> Entering new None chain...
===== 도구 호출 =====
사용된 도구 이름 : pdf_search
===== 도구 호출 =====

Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`


SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고

In [34]:
from langchain_teddynote.messages import AgentStreamParser

agent_stream_parser = AgentStreamParser()           # 각 단계별 출력을 위한 파서 생성

result = agent_executor.stream(                     # 질의에 대한 답변을 스트리밍으로 출력 요청
    {'input': '삼성전자가 자체 개발한 생성형 AI에 대한 정보를 문서에서 검색해서 알려주세요.'}
)

for step in result:
    agent_stream_parser.process_agent_steps(step)   # 중간 단계를 parser 를 사용하여 단계별로 출력



> Entering new None chain...
[도구 호출]
Tool: pdf_search
query: 삼성전자 생성형 AI
Log: 
Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`




Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`


SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비